<a href="https://colab.research.google.com/github/wanwanliang/Image_Processing_and_Deep_Learning/blob/master/code/Extract_Statistics_from_Unmasked_Region_with_Geotiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare environment and data

In [ ]:
!pip install geopandas
!pip install rasterio

In [ ]:
import google.colab
import numpy as np
import os
import pandas as pd
import geopandas as gpd
import rasterio as rio
from google.colab import drive
import tensorflow as tf
import glob
%matplotlib inline
from matplotlib import mlab 
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import PIL as pil

## Set work dir

In [ ]:
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/My Drive/UMN_Research/Data/wsr')

In [ ]:
os.chdir('/content/drive/My Drive/UMN_Research/Data/wsr/image_200_bb45/MCARI2')

## List all files

In [ ]:
tifs = glob.glob('*.tif')

In [ ]:
len(tifs)

In [ ]:
tifs[:5]

# Image exploring

## Histogram plot and percentile plot

In [ ]:
img = rio.open(tifs[0])
arys = img.read()
arys2 = np.moveaxis(arys, 0, -1)
arys2.shape

In [ ]:
arys3 = arys2.reshape((-1,1))
arys3.shape

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(arys3, bins=20)
plt.show()

In [ ]:
hist, bins = np.histogram(arys3, bins=20)
width = 0.7*(bins[1]-bins[0])
center = (bins[:-1]+bins[1:])/2
plt.bar(center, hist,align='center',width=width)
plt.show()

In [ ]:
pdd = pd.DataFrame(arys3)
print(pdd.head(5))
pdd.describe()

In [ ]:
import scipy.stats

# 100 values from a normal distribution with a std of 3 and a mean of 0.5
data = arys3[:,0]
counts, start, dx, _ = scipy.stats.cumfreq(data, numbins=20)
x = np.arange(counts.size) * dx + start
freq = counts/len(arys3[:,0])
plt.plot(x, freq, 'ro')
plt.xlabel('Value')
plt.ylabel('Cumulative Frequency')

plt.show()

In [ ]:
kwargs={'cumulative':True}
sns.distplot(arys3[:,0], hist_kws=kwargs, kde_kws=kwargs)

In [ ]:
p = np.array([0.0,2.5, 5.0, 10.0, 25.0, 50.0, 75.0, 90.0, 95.0, 97.5, 100.0])
perc = np.percentile(arys3, p)
perc

## Exploring image thresholding

In [ ]:
from skimage.filters import try_all_threshold
import skimage.filters as filters

fig, ax = try_all_threshold(arys2.reshape((30,31)), figsize=(10, 8), verbose=False)
plt.show()

In [ ]:
lst = ['otsu','li','yen','isodata','mean']

mint= min_threshold(arys2.reshape((30,31)), lst)

In [ ]:
binary = arys2.reshape((30,31)) > mint
plt.imshow(binary)
plt.show()

In [ ]:
template = img
ary2tif(binary, img, 'ts.tif')

# Image thresholding

## Define functions

### Image thresholding function

In [ ]:
def img_threshold(img, method=None):
  if method=='otsu':
    thre = filters.threshold_otsu(img)
  if method=='li':
    thre = filters.threshold_li(img)
  if method=='yen':
    thre = filters.threshold_yen(img)
  if method=='isodata':
    thre = filters.threshold_isodata(img)
  if method=='mean':
    thre = filters.threshold_mean(img)
    
  return(thre)

### Get minimum threshold

In [ ]:
def min_threshold(img, method_list):
  thre = []
  for method in lst:
    thre.append(img_threshold(img, method))

  return(min(thre))

### Image array to geotiff

In [ ]:
def ary2tif(img, img_template, nm):
  img = img.astype('float32')
  meta = img_template.meta.copy()
  #meta.update({'nodata': 999, 'dtype': 'float32', 'count':1})


  with rio.open(nm, 'w', **meta) as outf:
    outf.write(img, 1)

### Geotiff to numpy array

In [ ]:
def tif2ary(tif):

  raA = rio.open(tif)
  arys = raA.read()

  arys= arys.astype('float32')
  arys =np.moveaxis(arys, 0, -1)

  return(arys)

### Get image statistic for unmask region

In [ ]:
def img_stats(img_mlp, vi):

  vi_ary = tif2ary(vi)
  threshold = img_threshold(vi_ary, 'otsu')
  img_binary = vi_ary < threshold 
  img_binary = img_binary.reshape((img_binary.shape[0], img_binary.shape[1]))

  img_ary = tif2ary(img_mlp)
  img_ary[img_binary] = np.nan

  img_ary2 = np.concatenate(img_ary,0)
  
 
  fea_ls = []

  for i in range(img_ary2.shape[1]):
    
    des = describe(img_ary2[:,i], axis=0, nan_policy= 'omit')

    mean = des.mean
    var = des.variance

    fea_ls.append(mean)
    fea_ls.append(var)
  
  return(fea_ls)

In [ ]:
#img_binary = np.concatenate(img_binary,0)
img_binary = img_binary.reshape((img_binary.shape[0],-1))
img_binary.shape

In [ ]:
def vis_stats(vi_mlp, vi):

  vi_ary = tif2ary(vi)
  threshold = img_threshold(vi_ary, 'otsu')
  img_binary = vi_ary < threshold 
  img_binary = img_binary.reshape((img_binary.shape[0],img_binary.shape[1]))
  img_binary = np.concatenate(img_binary,0)
  #img_binary = img_binary.reshape((img_binary.shape[0],-1))

  vi_mlp[img_binary] = np.nan

  img_ary2 = vi_mlp
  
 
  fea_ls = []

  for i in range(img_ary2.shape[1]):
    
    des = describe(img_ary2[:,i], axis=0, nan_policy= 'omit')

    mean = des.mean
    var = des.variance

    fea_ls.append(mean)
    fea_ls.append(var)
  
  return(fea_ls)

In [ ]:
  vi_ary = tif2ary(vi)
  threshold = img_threshold(vi_ary, 'otsu')
  img_binary = vi_ary < threshold 
  img_binary = img_binary.reshape((img_binary.shape[0],img_binary.shape[1]))
  img_binary = np.concatenate(img_binary,0)
 # img_binary = img_binary.reshape((img_binary.shape[0],-1))
  img_binary.shape


In [ ]:
img_binary[:10,0]

In [ ]:
vis[img_binary]=np.nan

In [ ]:
vis.shape

### Calculate vegetation index

In [ ]:
def cal_vis(img):
  img_ary = tif2ary(img)

  r = img_ary[:,:,0]
  g = img_ary[:,:,1]
  b = img_ary[:,:,2]
  nir = img_ary[:,:,6]
  reg = img_ary[:,:,7]

  r2 = r.reshape((r.shape[0]*r.shape[1],-1))
  g2 = g.reshape((g.shape[0]*g.shape[1],-1))
  b2 = b.reshape((b.shape[0]*b.shape[1],-1))
  nir2 = nir.reshape((nir.shape[0]*nir.shape[1],-1))
  reg2 = reg.reshape((reg.shape[0]*reg.shape[1],-1))

  nv = (nir2-r2)/(nir2+r2)
  sr = nir2/r2
  osavi = 1.16*(nir2-r2)/(nir2+r2+0.16)
  gi = g2/r2
  nri = (g2-r2)/(g2+r2)
  
  arys= np.concatenate((nv,sr,osavi,gi,nri),axis=1)
  return(arys)

In [ ]:
r = img_ary[:,:,0]
g = img_ary[:,:,1]
b = img_ary[:,:,2]
nir = img_ary[:,:,6]
reg = img_ary[:,:,7]

r2 = r.reshape((r.shape[0]*r.shape[1],-1))
g2 = g.reshape((g.shape[0]*g.shape[1],-1))
b2 = b.reshape((b.shape[0]*b.shape[1],-1))
nir2 = nir.reshape((nir.shape[0]*nir.shape[1],-1))
reg2 = reg.reshape((reg.shape[0]*reg.shape[1],-1))


In [ ]:
print(r.shape)
print(r2.shape)

In [ ]:
nv = (nir2-r2)/(nir2+r2)
sr = nir2/r2
osavi = 1.16*(nir2-r2)/(nir2+r2+0.16)

In [ ]:
arys= np.concatenate((nv,sr,osavi),axis=1)
arys.shape


In [ ]:
nv.shape

# List images

In [ ]:
os.getcwd()

In [ ]:
os.chdir('/content/drive/My Drive/UMN_Research/Data/wsr/image_200_bb45')

In [ ]:
tifs = glob.glob("*.tif")
print(("Number of tifs is {}".format(len(tifs))))
tifs[:5]

In [ ]:
vis = glob.glob("./MCARI2/*.tif")
print("Number of VI is {}".format(len(vis)))
vis[:10]

# Extract mean and variance for each band 

In [ ]:
fea = None

for i in range(len(tifs)):

  mlp = tifs[i]
  vi = './MCARI2/' + mlp

  dt = img_stats(mlp, vi)
  dt = np.asarray(dt)
  dt = dt.reshape((16,-1))

  if fea is None:
    fea = dt
  else:
    fea = np.concatenate((fea, dt),axis=1)

# Extract mean and variance from vegetation index

In [ ]:
fea = None

for i in range(len(tifs)):

  mlp = tifs[i]
  vi = './MCARI2/' + mlp
  vis = cal_vis(mlp)

  dt = vis_stats(vis, vi)
  dt = np.asarray(dt)
  dt = dt.reshape((10,-1))

  if fea is None:
    fea = dt
  else:
    fea = np.concatenate((fea, dt),axis=1)

In [ ]:
fea.shape

In [ ]:
fea2 = np.transpose(fea)
fea2.shape

In [ ]:
fea_df = pd.DataFrame(fea2)
print(fea_df.shape)
fea_df.head

# Get image ID

In [ ]:
tifs[:10]

In [ ]:
nms=[]

[nms.append(txt.split("_")[4]) for txt in tifs]

In [ ]:
print(len(nms))
nms[:5]

In [ ]:
nms2 = np.asarray(nms)
nms2 = nms2.reshape((len(nms),-1))
nms_df = pd.DataFrame(nms2)
print(nms2.shape)
print(nms_df.shape)

In [ ]:
fea_df['ply_ID']=nms_df

In [ ]:
fea_df.shape

In [ ]:
fea_df.head(5)

# Write data

In [ ]:
#fea_df.to_csv("/BandStats.csv", index=False)

In [ ]:
fea_df.to_csv("/VIsStats.csv", index=False)